In [ ]:
import os
import yt_dlp
import numpy as np
import soundfile as sf
import pyloudnorm as pyln
from openpyxl import load_workbook
from pydub import AudioSegment
from mutagen.wave import WAVE

# === CONFIG ===
EXPORT_FOLDER = "procuced_wav_files"
TARGET_LUFS = -14.0
os.makedirs(EXPORT_FOLDER, exist_ok=True)

# Load song titles
def load_song_titles(xlsx_path):
    wb = load_workbook(xlsx_path)
    ws = wb.active
    return [row[0].value for row in ws.iter_rows(min_row=2) if row[0].value]

# Download highest quality audio
def download_best_audio(song_title):
    search_query = f"ytsearch10:{song_title}"
    print(f"🔍 Searching for best HQ match: {song_title}")

    try:
        with yt_dlp.YoutubeDL({'quiet': True}) as ydl:
            search_results = ydl.extract_info(search_query, download=False)["entries"]
    except Exception as e:
        print(f"❌ Search failed: {e}")
        return

    # Filter for 'HQ' or 'HD' in the title
    filtered = [entry for entry in search_results if entry and entry.get("title") and ("hq" in entry["title"].lower())]

    best_entry = filtered[0] if filtered else search_results[0]
    if not best_entry:
        print(f"❌ No video found for: {song_title}")
        return

    url = best_entry["webpage_url"]
    print(f"🎯 Selected: {best_entry['title']}")

    ydl_opts = {
        'format': 'bestaudio/best',
        'quiet': True,
        'outtmpl': f'{EXPORT_FOLDER}/{song_title}.%(ext)s',
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
    except Exception as e:
        print(f"❌ Failed to download '{song_title}': {e}")


# Convert to WAV and apply LUFS normalization
def convert_to_lufs_wav(song_title):
    raw_path = None
    for ext in ["webm", "m4a", "opus", "mp3"]:
        candidate = os.path.join(EXPORT_FOLDER, f"{song_title}.{ext}")
        if os.path.exists(candidate):
            raw_path = candidate
            break

    if not raw_path:
        print(f"❌ No downloaded audio found for: {song_title}")
        return

    print(f"🎼 Converting and normalizing: {song_title}")
    audio = AudioSegment.from_file(raw_path)
    wav_temp_path = os.path.join(EXPORT_FOLDER, f"{song_title}_temp.wav")
    audio.export(wav_temp_path, format="wav")

    # Load raw WAV into numpy
    y, sr = sf.read(wav_temp_path)
    if len(y.shape) > 1:  # stereo to mono
        y = y.mean(axis=1)

    # Measure loudness
    meter = pyln.Meter(sr)
    loudness = meter.integrated_loudness(y)
    print(f"📏 Original LUFS: {loudness:.2f}")

    # Normalize
    y_normalized = pyln.normalize.loudness(y, loudness, TARGET_LUFS)

    # Export normalized audio
    final_wav_path = os.path.join(EXPORT_FOLDER, f"{song_title}.wav")
    sf.write(final_wav_path, y_normalized, sr)
    os.remove(wav_temp_path)
    os.remove(raw_path)
    print(f"✅ LUFS-normalized WAV saved: {final_wav_path}")

    # Optional metadata
    add_metadata(final_wav_path, song_title)

# Add metadata to WAV
def add_metadata(filepath, song_title):
    artist = "Unknown Artist"
    if " - " in song_title:
        artist, song_title = song_title.split(" - ", 1)

    try:
        audio = WAVE(filepath)
        audio["title"] = song_title
        audio["artist"] = artist
        audio.save()
        print(f"🏷️ Metadata tagged: {song_title} by {artist}")
    except Exception as e:
        print(f"⚠️ Metadata tagging failed: {e}")

# Main
if __name__ == "__main__":
    titles = load_song_titles("songs.xlsx")
    for title in titles:
        download_best_audio(title)
        convert_to_lufs_wav(title)


🎯 Selected: Hurricane - I'm On To You (HD)
🎼 Converting and normalizing: Hurricane I am on To you  
📏 Original LUFS: -16.01
✅ LUFS-normalized WAV saved: procuced_wav_files\Hurricane I am on To you.wav
⚠️ Metadata tagging failed: 'Hurricane I am on To you' not a Frame instance
🔍 Searching for best HQ/HD match: Saraya In the shade of the Sun


c:\Users\PX\anaconda3\envs\yt_autotune\lib\site-packages\pyloudnorm\normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")


🎯 Selected: SARAYA - In the Shade of the Sun (1991)
🎼 Converting and normalizing: Saraya In the shade of the Sun
📏 Original LUFS: -16.97


c:\Users\PX\anaconda3\envs\yt_autotune\lib\site-packages\pyloudnorm\normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")


✅ LUFS-normalized WAV saved: procuced_wav_files\Saraya In the shade of the Sun.wav
⚠️ Metadata tagging failed: 'Saraya In the shade of the Sun' not a Frame instance
🔍 Searching for best HQ/HD match: Blue Murder - Valley Of The Kings
🎯 Selected: Blue Murder - Valley Of The Kings (HQ)
🎼 Converting and normalizing: Blue Murder - Valley Of The Kings
📏 Original LUFS: -19.10


c:\Users\PX\anaconda3\envs\yt_autotune\lib\site-packages\pyloudnorm\normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")


✅ LUFS-normalized WAV saved: procuced_wav_files\Blue Murder - Valley Of The Kings.wav
⚠️ Metadata tagging failed: 'Valley Of The Kings' not a Frame instance
🔍 Searching for best HQ/HD match: White Lion Hungry
